In [1]:
import menpo.io as mio
from menpo.visualize import print_progress
from menpo.landmark import labeller, face_ibug_68_to_face_ibug_68_trimesh
from menpofit.aam import HolisticAAM
from menpo.feature import fast_dsift
from menpofit.aam import LucasKanadeAAMFitter, WibergInverseCompositional
import numpy as np
import menpo

path_to_images = '/home/rshaydu/tensorfaces/helen/trainset/'
training_images = []
for img in print_progress(mio.import_images(path_to_images, verbose=True)):
    # convert to greyscale
    if img.n_channels == 3:
        img = img.as_greyscale()
    # crop to landmarks bounding box with an extra 20% padding
    img = img.crop_to_landmarks_proportion(0.2)
    # rescale image if its diagonal is bigger than 400 pixels
    d = img.diagonal()
    if d > 400:
        img = img.rescale(400.0 / d)
    # define a TriMesh which will be useful for Piecewise Affine Warp of HolisticAAM
    labeller(img, 'PTS', face_ibug_68_to_face_ibug_68_trimesh)
    # append to list
    training_images.append(img)

aam = HolisticAAM(training_images, group='face_ibug_68_trimesh', diagonal=150,
                  scales=(0.5, 1.0), holistic_features=fast_dsift, verbose=True,
                  max_shape_components=20, max_appearance_components=150)


fitter = LucasKanadeAAMFitter(aam, lk_algorithm_cls=WibergInverseCompositional,
                              n_shape=[5, 20], n_appearance=[30, 150])

Found 2000 assets, index the returned LazyList to import.
[====                ] 20% (403/2000) - 00:04:04 remaining                      

ValueError: Unexpected dtype (object) - normalisation range is unknown

In [ ]:
import os
import sys
from menpodetect import load_dlib_frontal_face_detector


# Load and convert to grayscale
image_folder = '/home/rshaydu/tensorfaces/FaceBase_png/'
img_files = list(filter(lambda x: ".png" in x, os.listdir(image_folder)))

# somewhere here a for loop starts

for img_file in img_files:
    img_path = image_folder+img_file

    print("Landmarking ", img_path)

    image = mio.import_image(img_path)
    image = image.as_greyscale()

    # Load detector
    detect = load_dlib_frontal_face_detector()

    # # Detect face
    bboxes = detect(image)
    initial_box = None
    if len(bboxes) > 0:
        initial_bbox = bboxes[0]
        print("dlib_frontal_face_detector found an initial box")
    else:
    # initial bbox
    # build box by hand
        adjacency_matrix = np.array([[0,1,0,0],
                                     [0,0,1,0],
                                     [0,0,0,1],
                                     [1,0,0,0]])

        points = None
        if 'vp0' in img_path or 'vp1' in img_path:
            points = np.array([[150,100],[500,100],[500,300],[150,300]]) # fine for looking right
        elif 'vp3' in img_path or 'vp4' in img_path:
            points = np.array([[120,50],[500,50],[500,250],[120,250]])
        elif 'vp0' in img_path:
            points = np.array([[120,75],[500,75],[500,275],[120,275]])

        if points == None:
            print("Error: incorrect finename", img_path)
            sys.exit(0)

        initial_bbox = menpo.shape.PointDirectedGraph(points, adjacency_matrix)
        print("Using a hardcoded initial box")

    # fit image
    result = fitter.fit_from_bb(image, initial_bbox, max_iters=200)

    # print result
    print(result)

    warped = fitter.warped_images(result.image, [result.final_shape, fitter.reference_shape])
    # warped[0].view()
    mio.export_image(warped[0],"/home/rshaydu/tensorfaces/FaceBase_warped/"+img_file)
    break